In [4]:
import requests
import json
import os
import numpy as np
from openai import OpenAI
from typing import List, Dict
import time
import datetime

In [5]:
from Last_Refresh import get_last_refresh_date, update_last_refresh_date
from BDD import save_embeddings_numpy, load_embeddings, append_embeddings


# API to fetch all VIE Id's

In [ ]:
def search_offers(limit=100, skip=0):
    # URL de l'API
    url = "https://civiweb-api-prd.azurewebsites.net/api/Offers/search"

    # Corps de la requête (payload)
    payload = {
        "limit": limit,
        "skip": skip,
        "latest": ["true"],
        "method": ["null"],
        "activitySectorId": [],
        "missionsTypesIds": ["1"],
        "missionsDurations": [],
        "gerographicZones": [],
        "countriesIds": [],
        "studiesLevelId": [],
        "companiesSizes": [],
        "specializationsIds": [],
        "entreprisesIds": [0],
        "missionStartDate": None,
        "query": None
    }

    # Headers
    headers = {
        "Content-Type": "application/json"
    }

    # Effectuer la requête POST
    response = requests.post(url, json=payload, headers=headers)
    # Vérifier le code de statut
    response.raise_for_status()
    # Afficher le résultat
    print(f"Code de statut: {response.status_code}")
        
    return response.json()

#search_offers(1,0)
    

# API to fetch all the data for each ids

In [ ]:
def get_offer_details(offer_id):
    url = f"https://civiweb-api-prd.azurewebsites.net/api/Offers/details/{offer_id}"
    # Headers (optionnel pour un GET simple)
    headers = {
        "Accept": "application/json"
    }
    # Effectuer la requête GET
    response = requests.get(url, headers=headers)
    # Vérifier le code de statut
    response.raise_for_status()
    print("   Récupération des données réussies")
    return response.json()

#get_offer_details(227934)

    

## Methods to compute application rate of the Job

In [ ]:
def enhance_offer_data(offer_data):
    """Améliore les données avec des métriques calculées"""
    candidates = offer_data.get('candidateCounter', 0)
    views = offer_data.get('viewCounter', 0)
    
    # Calcul du taux de postulation
    application_rate = 0
    if views > 0:
        application_rate = round((candidates / views) * 100, 1)
    
    # Catégorisation de la compétition
    competition_level = "FAIBLE"
    if application_rate > 10:
        competition_level = "ÉLEVÉE"
    elif application_rate > 5:
        competition_level = "MOYENNE"
    
    return {
        **offer_data,
        "application_rate": application_rate,  
        "competition_level": competition_level,  
        "candidates_count": candidates,
        "views_count": views
    }

## Methods to clean the data, (Used in the creating chunk method)

In [ ]:
def clean_offer_data(offer_data):
    print("   Nettoyage des données")
    """Nettoie et complète les données manquantes"""
    return {
        "reference": offer_data.get("reference", "N/A"),
        "organizationName": offer_data.get("organizationName", "Entreprise non spécifiée"),
        "missionTitle": offer_data.get("missionTitle", "Titre non spécifié"),
        "missionDescription": offer_data.get("missionDescription", "Description non disponible"),
        "missionProfile": offer_data.get("missionProfile", "Profil non spécifié"),
        "countryNameEn": offer_data.get("countryNameEn", "Pays non spécifié"),
        "cityNameEn": offer_data.get("cityNameEn", "Ville non spécifiée"),
        "activitySectorN1": offer_data.get("activitySectorN1", "Secteur non spécifié"),
        "missionDuration": offer_data.get("missionDuration", "Durée non spécifiée"),
        "indemnite": offer_data.get("indemnite", "Non spécifié"),
        "missionStartDate": offer_data.get("missionStartDate", "Date non spécifiée"),
        "creationDate": offer_data.get("creationDate","Date non spécifiée"),
        "contactEmail": offer_data.get("contactEmail", "Email non disponible"),
    }

# Creating chunks method

We are dividing our chunks into content and metadata.

We will only apply embedding on our content.

Metadata will allow to use some filters when fetching the embeddings in the ChromaDB database

In [ ]:

def create_chunks_for_rag(offer_data):
    """Crée 2 chunks optimisés pour le RAG"""
    chunks = []

    # 1. Nettoyer les données brutes
    cleaned_offer = clean_offer_data(offer_data)

    # 2. Calculer le taux de postulation à l'offre
    enhanced_data = enhance_offer_data(offer_data)

    # 2. Extraire les métadonnées depuis les données nettoyées
    common_metadata = {
        "offer_reference": cleaned_offer.get("reference"),  
        "company": cleaned_offer.get("organizationName"),
        "title": cleaned_offer.get("missionTitle"),
        "country": cleaned_offer.get("countryNameEn"),
        "city": cleaned_offer.get("cityNameEn"),
        "sector": cleaned_offer.get("activitySectorN1"),
        "duration_months": cleaned_offer.get("missionDuration"),
        "salary_eur": cleaned_offer.get("indemnite"),
        "start_date": cleaned_offer.get("missionStartDate"),
        "creation_date": cleaned_offer.get("creationDate"),
        "contact_email": cleaned_offer.get("contactEmail"),
        # Rest is comming from enhanced_data
        "application_rate": enhanced_data["application_rate"],
        "competition_level": enhanced_data["competition_level"], 
        "candidates_count": enhanced_data["candidates_count"],
        "views_count": enhanced_data["views_count"]
    }

    
    
    # CHUNK 1: Description de la mission
    chunk1_content = f"""Offre VIE {offer_data.get('reference')} - {offer_data.get('missionTitle')}
    Entreprise: {offer_data.get('organizationName')}
    Localisation: {offer_data.get('cityNameEn')}, {offer_data.get('countryNameEn')}
    Secteur d'activité: {offer_data.get('activitySectorN1')}
    Durée: {offer_data.get('missionDuration')} mois
    Indemnité: {offer_data.get('indemnite')} € par mois
    Description de la mission: {offer_data.get('missionDescription', 'Non spécifiée')}"""

    new_chunk = {
        "content": chunk1_content,
        "metadata": {
            **common_metadata,
            "chunk_type": "mission_description",
            "chunk_id": f"{offer_data.get('id')}_description"
        }
    }

    print(f"   Taille du chunk de Description (caractères) : {len(new_chunk['content'])}")
    chunks.append(new_chunk)
    
    # CHUNK 2: Profil recherché
    chunk2_content = f"""Offre VIE {offer_data.get('reference')} - {offer_data.get('missionTitle')}
    Entreprise: {offer_data.get('organizationName')}
    Localisation: {offer_data.get('cityNameEn')}, {offer_data.get('countryNameEn')}
    Secteur d'activité: {offer_data.get('activitySectorN1')}
    Profil recherché: {offer_data.get('missionProfile', 'Non spécifié')}"""

    new_chunk2 = {
        "content": chunk2_content,
        "metadata": {
            **common_metadata,
            "chunk_type": "candidate_profile",
            "chunk_id": f"{offer_data.get('id')}_profile"
        }
    }

    chunks.append(new_chunk2)
    print(f"   Taille du chunk de Profil (caractères) : {len(new_chunk2['content'])}")
    
    return chunks

# Create the embeding of the chunks

In [6]:
# Configuration OpenAI
api_key = os.environ.get("API_KEY_OPENAI")
client = OpenAI(api_key=api_key)

###  Method to embed the question that we will ask the RAG

In [7]:
def get_text_embedding(text: str) -> List[float]:
    """
    Crée un embedding pour un texte avec OpenAI
    
    Args:
        text: Le texte à embedder
        
    Returns:
        Liste de floats représentant l'embedding
    """
    response = client.embeddings.create(
        input=text,
        model="text-embedding-3-small"  # 1536 dimensions, $0.02/1M tokens
    )
    return response.data[0].embedding

###  Method to create our embeddings dataset

We are using batch to optimize API calls and reduce latency. 

This approach processes multiple texts in a single request instead of making individual calls for each one.

In [ ]:
def create_embeddings_batch(chunks: List[Dict], batch_size: int = 100) -> List[Dict]:
    """
    Crée les embeddings pour tous les chunks avec traitement par batch
    
    Args:
        chunks: Liste des chunks à embedder
        batch_size: Nombre de chunks à traiter par batch (max 100 pour OpenAI)
        
    Returns:
        Liste des chunks enrichis avec leurs embeddings
    """
    enriched_chunks = []
    
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i + batch_size]
        print(f"Traitement du batch {i//batch_size + 1} ({len(batch)} chunks)...")
        
        # Extraire les contenus textuels
        texts = [chunk["content"] for chunk in batch]
        
        # Créer les embeddings en batch (plus efficace)
        response = client.embeddings.create(
            input=texts,
            model="text-embedding-3-small"
        )
        
        # Associer chaque embedding à son chunk
        for j, chunk in enumerate(batch):
            enriched_chunk = chunk.copy()
            enriched_chunk["embedding"] = response.data[j].embedding
            enriched_chunks.append(enriched_chunk)
        
        # Rate limiting (optionnel mais recommandé)
        time.sleep(0.1)
    
    print(f"✅ {len(enriched_chunks)} embeddings créés")
    return enriched_chunks




# Creation of the Pipeline : create the dataset embeddings

In [ ]:
print("="*80)
print("PIPELINE RAG - OFFRES VIE")
print("="*80)

# ÉTAPE 1: Rechercher les offres
print("\n[1/5] Recherche des offres VIE...")
offers_response = search_offers(limit=10000)  # Ajustez la limite selon vos besoins

# La structure de retour peut varier, adaptez selon l'API
# Supposons que l'API retourne une liste d'IDs ou d'objets simplifiés
offer_ids = []
offer_ids = [item.get("id") for item in offers_response["result"] if item.get("id") is not None]

print(f"✅ {len(offer_ids)} offres trouvées")

# ÉTAPE 2: Récupérer les détails et créer les chunks
print("\n[2/5] Récupération des détails et création des chunks...")
all_chunks = []
    

for i, offer_id in enumerate(offer_ids, 1):  
    try:
        print(f"  Traitement offre {i}/{len(offer_ids)}: {offer_id}")
        offer_details = get_offer_details(offer_id)
        chunks = create_chunks_for_rag(offer_details)
        all_chunks.extend(chunks)
        time.sleep(0.2)  # Rate limiting pour l'API Civiweb
    except Exception as e:
        print(f"  ⚠️  Erreur pour l'offre {offer_id}: {e}")
        continue
    
    print(f"✅ {len(all_chunks)} chunks créés\n")

# ÉTAPE 3: Créer les embeddings
print("\n[3/5] Création des embeddings OpenAI...")
chunks_with_embeddings = create_embeddings_batch(all_chunks)


print("\n[4/5] Sauvegarde des nouvelles données...")
save_embeddings_numpy(chunks_with_embeddings, "vie_embeddings.npz")

# We save the current date
update_last_refresh_date()

# ÉTAPE 5: Statistiques
print("\n[5/5] Statistiques finales")
print("="*80)
print(f"Nombre de nouvelles offres traitées: {len(offer_ids)}")
print(f"Nombre de nouveaux chunks: {len(chunks_with_embeddings)}")
print(f"Dimension des nouveaux embeddings: {len(chunks_with_embeddings[0]['embedding'])}")

# Calculer la taille totale
total_tokens = sum(len(c['content']) // 4 for c in chunks_with_embeddings)
print(f"Tokens estimés: ~{total_tokens:,}")
print(f"Coût estimé: ~${(total_tokens / 1_000_000) * 0.02:.4f}")
print("="*80)


# Creation of the Pipeline : Add new embeddings in the curent dataset

In [ ]:
print("="*80)
print("PIPELINE RAG - Nouvelles OFFRES VIE")
print("="*80)

# ÉTAPE 1: Rechercher les offres
print("\n[1/5] Recherche des offres VIE...")
offers_response = search_offers(limit=100000)  # Ajustez la limite selon vos besoins

last_refresh_date = get_last_refresh_date()
print(f"📅 Dernier rafraîchissement: {last_refresh_date}")

offer_ids = []
offer_ids = [
    item.get("id") for item in offers_response["result"] 
    if (item.get("id") is not None and 
        datetime.fromisoformat(item.get("creationDate")) > last_refresh_date)
]

print(f"✅ {len(offer_ids)} nouvelles offres depuis le dernier rafraîchissement")
# Mettre à jour Last_refresh après le traitement
update_last_refresh_date()

# ÉTAPE 2: Récupérer les détails et créer les chunks
print("\n[2/5] Récupération des détails et création des chunks...")
all_chunks = []
    

for i, offer_id in enumerate(offer_ids, 1):  
    try:
        print(f"  Traitement offre {i}/{len(offer_ids)}: {offer_id}")
        offer_details = get_offer_details(offer_id)
        chunks = create_chunks_for_rag(offer_details)
        all_chunks.extend(chunks)
        time.sleep(0.2)  # Rate limiting pour l'API Civiweb
    except Exception as e:
        print(f"  ⚠️  Erreur pour l'offre {offer_id}: {e}")
        continue
    
    print(f"✅ {len(all_chunks)} chunks créés\n")

# ÉTAPE 3: Créer les embeddings
print("\n[3/5] Création des embeddings OpenAI...")
chunks_with_embeddings = create_embeddings_batch(all_chunks)


print("\n[4/5] Sauvegarde des données...")
save_embeddings_numpy(chunks_with_embeddings, "vie_embeddings.npz")


# ÉTAPE 5: Statistiques
print("\n[5/5] Statistiques finales")
print("="*80)
print(f"Nombre d'offres traitées: {len(offer_ids)}")
print(f"Nombre de chunks: {len(chunks_with_embeddings)}")
print(f"Dimension des embeddings: {len(chunks_with_embeddings[0]['embedding'])}")

# Calculer la taille totale
total_tokens = sum(len(c['content']) // 4 for c in chunks_with_embeddings)
print(f"Tokens estimés: ~{total_tokens:,}")
print(f"Coût estimé: ~${(total_tokens / 1_000_000) * 0.02:.4f}")
print("="*80)


# Now let's ask a question and embed it !!!

In [8]:
#question = "Je Cherche un VIE de minimum 1 an, je suis orientée en Data et IA mais j'aime aussi faire du développement web. Peux tu me donner les meilleurs offres pour moi. Sachant que je cherche une offre pour Octobre 2025"
question = "Je Cherche un VIE de minimum 1 an, je suis orientée en Data et IA mais j'aime aussi faire du développement web. Peux tu me donner les meilleurs offres pour moi. Si possible priorise les offres en corée du sud"
# question = "Je Cherche un VIE en Asie de l'Est  pour faire de la data ou de l'IA"


In [9]:
question_embeddings = np.array([get_text_embedding(question)])

# Comparison bewtween our embedding database and our embedding question

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
def find_closest_embedding(
    question_embeddings: np.array,
    text_embeddings: np.ndarray,
    top_n: int = 3,
) -> tuple[np.ndarray, np.ndarray]:

    # Calculer les similarités avec tous les embeddings
    similarites = cosine_similarity(question_embeddings, text_embeddings)[0]

    # Récupérer les top_n index et scores
    top_index = np.argsort(similarites)[-top_n:][::-1]  # Tri décroissant
    top_scores = similarites[top_index]

    return top_scores, top_index

In [13]:
loaded_embeddings = load_embeddings("vie_embeddings.npz")
text_embeddings = loaded_embeddings['embeddings']
top_scores, top_index = find_closest_embedding(question_embeddings, text_embeddings, 10)

top_results = []

print("Top 10 embeddings les plus proches:\n ")
for score, idx in zip(top_scores, top_index):
    print(f"- Embedding {idx}: Score = {score:.4f}")
    #print(f"metadata : {loaded_embeddings['metadata'][idx]}")
    #print(f"contents : {loaded_embeddings['contents'][idx]}")
    top_results.append(loaded_embeddings['contents'][idx])

    # loaded_embeddings['metadata'][idx] PAS UTILISER POUR L'INSTANT

Top 10 embeddings les plus proches:
 
- Embedding 775: Score = 0.6460
- Embedding 779: Score = 0.6450
- Embedding 677: Score = 0.6409
- Embedding 643: Score = 0.6407
- Embedding 645: Score = 0.6396
- Embedding 2029: Score = 0.6384
- Embedding 1809: Score = 0.6329
- Embedding 14: Score = 0.6293
- Embedding 1573: Score = 0.6274
- Embedding 2451: Score = 0.6270


# We Use a LLM to answer our question using the 10 closest embedding

In [12]:

# Concaténer les top-N segments en un seul contexte
context = "\n".join(top_results)
prompt = f"Contexte :\n{context}\n\nQuestion : {question}\nRéponse :"


response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Tu es un assistant spécialisé dans la recherche d'information à partir de documents fournis. Tes réponses doivent absolument provenir du contexte fourni."},
        {"role": "user", "content": prompt}
    ]
)

print(response.choices[0].message.content)


Il n'y a pas d'offres de VIE situées en Corée du Sud dans le contexte fourni. Cependant, voici quelques offres qui pourraient correspondre à tes intérêts en Data, IA, et développement web, chacune avec une durée de VIE d'au moins 1 an :

1. **Offre VIE VIE230265 - DATA SCIENTIST (H/F)**
   - **Entreprise:** OPTIMETRIKS
   - **Localisation:** DAKAR, SENEGAL
   - **Profil recherché:**
     - Solide intérêt pour le deep learning et la gestion des données
     - Maîtrise de Python et SQL
     - Connaissance des bibliothèques ML/DL comme TensorFlow/PyTorch, Scikit-learn, XGBoost
     - Disponible pour au moins 12 mois

2. **Offre VIE VIE229576 - DATA ENGINEER DEVELOPPER W/ FRENCH (H/F)**
   - **Entreprise:** EXTIA
   - **Localisation:** BUCHAREST, ROMANIA
   - **Profil recherché:**
     - Connaissance en ETL, Data mining, Machine learning, Big data
     - Maîtrise des bases de l’analyse statistique
     - Aptitude à rédiger des scripts en Python et/ou R
     - Familiarité avec l’environneme